In [2]:
from graphes import *
g=graphe_oriente(100,10)
#1) sous forme de tableau sommet:[liste de successeurs ]
#1.1) successeurs de 0 : [2, 8, 22, 28, 42, 48, 62, 68, 82, 88]
#1.2) successeurs de 67: [3, 20, 33, 50, 71, 88]
#2)
def degres(graph:dict,sommet:int):
    return len(graph[sommet])

def test_degres():
    test_graph=graphe_oriente(2,3)
    assert degres(test_graph,0)==1
    assert degres(test_graph,1)==0
    test_graph=graphe_oriente(8,3)
    assert degres(test_graph,0)==3
    assert degres(test_graph,1)==1
#3)
def sommets(graph:dict):
    return list(graph.keys())

def test_sommets():
    test_graph=graphe_oriente(2,3)
    assert len(sommets(test_graph))==2
    test_graph=graphe_oriente(8,3)
    assert len(sommets(test_graph))==8

#4)
def arcs(graph:dict):
    for start,ends in graph.items():
        for end in ends:
            yield (start,end)


#si besoin d'une liste et non d'un iterateur
def list_arcs(graph:dict):
    liste=[]
    for start,ends in graph.items():
        for end in ends:
            liste.append((start,end))
    return liste

def test_arcs():
    test_graph=graphe_oriente(2,3)
    assert len(list_arcs(test_graph))==1
    test_graph=graphe_oriente(8,3)
    assert len(list_arcs(test_graph))==15

#5
def describe(graph:dict):
    return (len(sommets(graph)),len(arcs(graph)))

def test_describe():
    test_graph=graphe_oriente(2,3)
    assert describe(test_graph)==(2, 1)
    test_graph=graphe_oriente(8,3)
    assert describe(test_graph)==(8, 15)
#6
def mirroir(graph:dict):
    mirror_edge={key:[] for key in graph.keys()}
    for start,ends in graph.items():
        for end in ends:
            mirror_edge[end].append(start)
    return mirror_edge

def test_mirroir():
    test_graph=graphe_oriente(100,30)
    assert mirroir(mirroir(test_graph))==test_graph
#7
def degres_entrant_sommet(graph:dict,sommet:int):
    return sum([1 if sommet in predecesseurs else 0 for predecesseurs in graph.values()])
    # return len(mirror(graph)[somment])

def test_degres_entrant_sommet():
    test_graph=graphe_oriente(10,2)
    assert degres_entrant_sommet(test_graph,0)==5
#8
def sous_graph(graph:dict,sommets:list[int]):
    sous_graph_tmp={key:[] for key in graph.keys() if key in sommets}
    for start,ends in graph.items():
        if start in sommets:
            for end in ends:
                if end in sommets:
                    sous_graph_tmp[start].append(end)
    return sous_graph_tmp

def test_sous_graph():
    test_graph=graphe_oriente(10,2)
    assert sous_graph(test_graph,[0,1])=={0: [], 1: [0]}



In [19]:
def Accessibles1(graphe:dict,x:int):
    liste_accessibles={x}
    for y in graphe.get(x,[]):
        sommets_atteignables=Accessibles1(graphe,y)
        liste_accessibles=liste_accessibles.union(sommets_atteignables)
    return liste_accessibles

In [20]:
graphe={0:[1,3],1:[],2:[0,3],3:[]}
print(Accessibles1(graphe,2))

{0, 1, 2, 3}


In [21]:
graphe={0:[1,3],1:[0],2:[0,3],3:[]}
print(Accessibles1(graphe,2))

RecursionError: maximum recursion depth exceeded while calling a Python object

0<->1 donc il fait une recurrence infinie

In [26]:
def Accessibles2(graphe:dict,x:int,liste_accessibles:set=set()):
    liste_accessibles.add(x) #corrigé sinon le premier sommet n'est pas mis
    for y in graphe.get(x,[]):
        if not y in liste_accessibles:
            liste_accessibles=Accessibles2(graphe,y,liste_accessibles)
    return liste_accessibles

In [27]:
graphe={0:[1,3],1:[0],2:[0,3],3:[]}
print(Accessibles2(graphe,2))

{0, 1, 2, 3}


In [1]:
def Accessibles3(graphe:dict,x:int,liste_accessibles:set=set(),chemins:list[tuple]=[]):
    liste_accessibles.add(x) #corrigé sinon le premier sommet n'est pas mis
    for y in graphe.get(x,[]):
        if not y in liste_accessibles:
            chemins.append((x,y))
            #liste_accessibles.add(y)
            (liste_accessibles,chemins)=Accessibles3(graphe,y,liste_accessibles,chemins)
    return (liste_accessibles,chemins)

graphe={0:[1,3,4,2],1:[0],2:[0,1],3:[4],4:[3]}
liste_accessibles,chemins=Accessibles3(graphe,0)
print(liste_accessibles)
chemins

{0, 1, 2, 3, 4}


[(0, 1), (0, 3), (3, 4), (0, 2)]

In [2]:
def reconstruire_chemin(start,end,_chemins)->list:
    if start==end:
        return [start]
    for tuple in _chemins:
        if tuple[1]!=end:continue
        tmp=reconstruire_chemin(start,tuple[0],_chemins)
        if tmp:
            return [*tmp,end]

reconstruire_chemin(0,4,chemins)

[0, 3, 4]

In [112]:
def Accessible(graphe,start,end):
    liste_accessibles,chemins=Accessibles3(graphe,start)
    if not end in liste_accessibles:
        return
    return reconstruire_chemin(start,end,chemins)

graphe={0:[1,3,4,2],1:[0],2:[0,1],3:[4],4:[3]}
bon_path=Accessible(graphe,0,4)
bon_path

[0, 3, 4]

In [113]:
def Verifier_chemin(graphe,start,end,_chemins):
    bon=(start==_chemins[0])
    last_somment=start
    for sommet in _chemins[1:]:
        if sommet in graphe[last_somment]:
            last_somment=sommet
        else:
            bon=False
    return bon and last_somment==end

false_path=[0,2,4]
assert Verifier_chemin(graphe,0,4,bon_path)
assert not Verifier_chemin(graphe,0,4,false_path)